In [ ]:
!cp "/content/drive/MyDrive/Master MVA/ALTEGRAD/Data Challenge ALTEGRAD/Files/graph_labels.txt" ./
#!cp "/content/drive/MyDrive/Master MVA/ALTEGRAD/Data Challenge ALTEGRAD/Files/edge_attributes.txt" ./
#!cp "/content/drive/MyDrive/Master MVA/ALTEGRAD/Data Challenge ALTEGRAD/Files/edgelist.txt" ./
#!cp "/content/drive/MyDrive/Master MVA/ALTEGRAD/Data Challenge ALTEGRAD/Files/graph_indicator.txt" ./
#!cp "/content/drive/MyDrive/Master MVA/ALTEGRAD/Data Challenge ALTEGRAD/Files/node_attributes.txt" ./
!cp "/content/drive/MyDrive/Master MVA/ALTEGRAD/Data Challenge ALTEGRAD/Files/sequences.txt" ./

In [ ]:
# Opção treinar do zero 
#https://www.kaggle.com/code/danofer/deep-protein-sequence-family-classification

In [6]:
!pip3 install -U pip > /dev/null
!pip3 install -U bio_embeddings[all] > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.
confection 0.0.3 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.6 which is incompatible.


In [7]:
!pip install bio-embeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import csv
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Read sequences
sequences = list()
with open('sequences.txt', 'r') as f:
    for line in f:
        sequences.append(line[:-1])

# Split data into training and test sets
sequences_train = list()
sequences_test = list()
proteins_test = list()
y_train = list()
with open('graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            proteins_test.append(t[0])
            sequences_test.append(sequences[i])
        else:
            sequences_train.append(sequences[i])
            y_train.append(int(t[1][:-1]))

# Map sequences to 
#vec = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
#X_train_sequence = vec.fit_transform(sequences_train)
#X_test_sequence = vec.transform(sequences_test)

In [8]:
from bio_embeddings.embed import ProtTransBertBFDEmbedder
embedder_model = ProtTransBertBFDEmbedder()
batch_size = 16

ImportError: ignored

In [ ]:
from tqdm import tqdm
X_train_sequence = []

for seq in tqdm(sequences_train):
    # do something with x
  #for i in range(0, len(sequences_train)):
  #seq_batch = sequences_train[i]#:min(len(sequences_train), i+batch_size)]
  embedding = embedder_model.embed(seq)
  X_train_sequence.append(embedding)

In [ ]:
X_train_sequence[0]

array([[ 0.04346859,  0.09442946, -0.01711776, ...,  0.13002351,
         0.03772632,  0.04799961],
       [ 0.05976252, -0.08812496,  0.00829028, ...,  0.00465693,
         0.00127334, -0.044973  ],
       [-0.11730403, -0.05469667,  0.09317888, ...,  0.00811603,
        -0.00057063,  0.02693714],
       ...,
       [ 0.05833338, -0.08616583,  0.01873964, ..., -0.03190076,
        -0.03273055, -0.13629143],
       [ 0.05115892, -0.03320181,  0.07323511, ..., -0.08184524,
        -0.05324489, -0.05401745],
       [ 0.2221749 , -0.04878725, -0.00125229, ...,  0.0156033 ,
        -0.00231342, -0.07513644]], dtype=float32)

In [ ]:
X_train_sequence[0].shape

(185, 1024)

In [ ]:
A = X_train_sequence[0]
B = np.pad(A, ((0, 989 - len(X_train_sequence[0])), (0,0)), 'constant')
B


array([[ 0.04346859,  0.09442946, -0.01711776, ...,  0.13002351,
         0.03772632,  0.04799961],
       [ 0.05976252, -0.08812496,  0.00829028, ...,  0.00465693,
         0.00127334, -0.044973  ],
       [-0.11730403, -0.05469667,  0.09317888, ...,  0.00811603,
        -0.00057063,  0.02693714],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [ ]:
B.shape

(989, 1024)

In [ ]:
X_test_sequence = []
for seq in tqdm(sequences_test):
  embedding = embedder_model.embed(seq)
  X_test_sequence.append(embedding)

100%|██████████| 1223/1223 [01:44<00:00, 11.66it/s]


In [ ]:
import pickle
with open('X_train_sequence_ProtTransBertBFDEmbedder.pkl', 'wb') as f:
    pickle.dump(X_train_sequence, f)

In [ ]:
with open('X_test_sequence_ProtTransBertBFDEmbedder.pkl', 'wb') as f:
    pickle.dump(X_test_sequence, f)

In [ ]:
sizes = []
for elem in sequences_train:
  sizes.append(len(elem))
sizes = np.array(sizes)

print(np.mean(sizes))
print(np.max(sizes))

258.14279869067104
989


In [ ]:
np.std(sizes)

162.24346887570542

In [ ]:
sequences[1]

'RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMDRYKTIIKKVGEKLCVEPAVIAGIISRESHAGKVLKNGWGDRGNGFGLMQVDKRSHKPQGTWNGEVHITQGTTILINFIKTIQKKFPSWTKDQQLKGGISAYNAGAGNVRSYARMDIGTTHDDYANDVVARAQYYKQHGY'

In [ ]:
input_seq_dim = X_train_sequence.shape[1]

In [ ]:
import csv
import time
import numpy as np
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

def load_data(): 
    """
    Function that loads graphs
    """  
    graph_indicator = np.loadtxt("graph_indicator.txt", dtype=np.int64)
    _,graph_size = np.unique(graph_indicator, return_counts=True)
    
    edges = np.loadtxt("edgelist.txt", dtype=np.int64, delimiter=",")
    edges_inv = np.vstack((edges[:,1], edges[:,0]))
    edges = np.vstack((edges, edges_inv.T))
    s = edges[:,0]*graph_indicator.size + edges[:,1]
    idx_sort = np.argsort(s)
    edges = edges[idx_sort,:]
    edges,idx_unique =  np.unique(edges, axis=0, return_index=True)
    A = sp.csr_matrix((np.ones(edges.shape[0]), (edges[:,0], edges[:,1])), shape=(graph_indicator.size, graph_indicator.size))
    
    x = np.loadtxt("node_attributes.txt", delimiter=",")
    edge_attr = np.loadtxt("edge_attributes.txt", delimiter=",")
    edge_attr = np.vstack((edge_attr,edge_attr))
    edge_attr = edge_attr[idx_sort,:]
    edge_attr = edge_attr[idx_unique,:]
    
    adj = []
    features = []
    edge_features = []
    idx_n = 0
    idx_m = 0
    for i in range(graph_size.size):
        adj.append(A[idx_n:idx_n+graph_size[i],idx_n:idx_n+graph_size[i]])
        edge_features.append(edge_attr[idx_m:idx_m+adj[i].nnz,:])
        features.append(x[idx_n:idx_n+graph_size[i],:])
        idx_n += graph_size[i]
        idx_m += adj[i].nnz

    return adj, features, edge_features

def normalize_adjacency(A):
    """
    Function that normalizes an adjacency matrix
    """
    n = A.shape[0]
    A += sp.identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D = sp.diags(inv_degs)
    A_normalized = D.dot(A)

    return A_normalized

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """
    Function that converts a Scipy sparse matrix to a sparse Torch tensor
    """
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


class GNN(nn.Module):
    """
    Simple message passing model that consists of 2 message passing layers
    and the sum aggregation function
    """
    def __init__(self, input_dim, input_seq_dim, hidden_dim, dropout, n_class):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, n_class)
        self.fc_seq1 = nn.Linear(input_seq_dim, 2*hidden_dim)
        self.fc_seq2 = nn.Linear(2*hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(2*hidden_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_in, adj, idx, seq):

        #print(seq.shape)
        # first message passing layer
        x = self.fc1(x_in)
        x = self.relu(torch.mm(adj, x))
        x = self.dropout(x)

        # second message passing layer
        x = self.fc2(x)
        x = self.relu(torch.mm(adj, x))
        
        # sum aggregator
        idx = idx.unsqueeze(1).repeat(1, x.size(1))
        out_g = torch.zeros(torch.max(idx)+1, x.size(1)).to(x_in.device)
        out_g = out_g.scatter_add_(0, idx, x)
        
        # batch normalization layer
        out_g = self.bn1(out_g)

        # mlp to produce output
        out_g = self.relu(self.fc3(out_g))
        out_g = self.dropout(out_g)

        # Processing of the sequence
        out_seq = self.relu(self.fc_seq1(seq))
        out_seq = self.dropout(out_seq)
        out_seq = self.relu(self.fc_seq2(out_seq))

        out_seq = self.bn2(out_seq)

        #print(out_g.shape)
        #print(out_seq.shape)

        # Merging both
        out = self.relu(self.fc_out(torch.cat((out_g, out_seq),1)))
        out = self.dropout(out)

        #print(torch.cat((out_g, out_seq),1).shape)

        # Produce logits
        out = self.fc4(out)

        return F.log_softmax(out, dim=1)


In [ ]:
class RNN_model(nn.Module):
    """
    Simple message passing model that consists of 2 message passing layers
    and the sum aggregation function
    """
    def __init__(self, input_dim, input_seq_dim, hidden_dim, dropout, n_class):
        super(RNN_model, self).__init__()
        self.rnn = nn.LSTM(input_size=1024, hidden_size=512, batch_first=True, bidirectional=False) 
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, n_class)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        out, (h_n, c_n) = self.rnn(x)
        #print(x.shape)
        #print(out.shape)
        #print(h_n.shape)
        #print(c_n.shape)
        #print("proximos")

        #out = self.relu(self.fc1(out[:,-1,:]))
        #out = torch.cat((h_n[0], h_n[1]), dim=1)
        #out = h_n[0]
        out = out.sum(dim=1)
        out = self.relu(self.fc1(out))
        #print(out.shape)
        out = self.relu(self.fc2(out))
        #print(out.shape)
        out = self.fc3(out)
        #print(out.shape)

        return F.log_softmax(out, dim=1)

In [ ]:
class CharCNN(nn.Module):
    
    def __init__(self, input_dim, input_seq_dim, hidden_dim, drop_p, n_classes):
        super().__init__()
        # params: "n_" means dimension
        self.n_vocab = n_vocab     # number of unique words in vocabulary 
        self.hidden_dim = hidden_dim   # number of hidden nodes in LSTM
        
        #self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=n_embed, padding_idx=0)
        self.conv1 = nn.Sequential(nn.Conv1d(1024, 256, kernel_size=7, padding=0), nn.ReLU(),nn.MaxPool1d(3))
        self.conv2 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=7, padding=0), nn.ReLU(),nn.MaxPool1d(3))
        self.conv3 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0), nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0), nn.ReLU(), nn.MaxPool1d(4))
        self.fc1 = nn.Sequential(nn.Linear(256, hidden_dim), nn.Dropout(drop_p))
        self.fc2 = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.Dropout(drop_p))
        self.fc3 = nn.Linear(hidden_dim, n_classes)

        self.dropout = nn.Dropout(drop_p)
        
        
    def forward(self, input_embed):
        #print(input_chars.shape)
        #embed = self.embedding(input_chars)
        #print(embed.shape)
        input_embed = embed.transpose(1, 2)
        output = self.conv1(embed)
        #print(output.shape)
        output = self.conv2(output)
        #print(output.shape)
        output = self.conv3(output)
        #print(output.shape)
        output = self.conv4(output)
        #print(output.shape)

        #output = output.view(output.size(0), -1)
        output = output.mean(dim=-1)
        output = self.fc1(output)
        #print(output.shape)
        output = self.fc2(output)
        #print(output.shape)
        output = self.fc3(output)
        #print(output.shape)

        return F.log_softmax(output, dim=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_sequence, X_val_sequence, y_train, y_val = train_test_split(X_train_sequence, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=1, min_delta=1, file_name="model.pt"):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf
        self.file_name = file_name

    def early_stop(self, model, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
            print(f"Saving best model at: '{self.file_name}'")
            torch.save(model.state_dict(), self.file_name)
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
#early_stopper = EarlyStopper(patience=3, min_delta=10)
#torch.save(model.state_dict(), 'best-model-parameters.pt') # official 

# To load
#the_model = TheModelClass(*args, **kwargs)
#the_model.load_state_dict(torch.load(PATH))

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
# Initialize device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Hyperparameters
epochs = 100 #50
batch_size = 64
n_hidden = 64
n_input = 86
dropout = 0.2
learning_rate = 1e-2 #0.001
n_class = 18

# Compute number of training and test samples
N_train = len(X_train_sequence)
N_val = len(X_val_sequence)
#N_test = len(adj_test)

# Initializes model and optimizer
model = CharCNN(n_input, 0, n_hidden, dropout, n_class).to(device)

optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=5, min_delta=0.1, file_name="merged-model.pt")

# Train model
for epoch in range(epochs):
    t = time.time()
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    # Iterate over the batches
    for i in range(0, N_train, batch_size):
        X_sequence_batch = list()
        y_batch = list()
        
        # Create tensors
        for j in range(i, min(N_train, i+batch_size)):
            X_sequence_batch.append(torch.FloatTensor(X_train_sequence[j]))
            y_batch.append(y_train[j])
            
        X_sequence_batch = pad_sequence(X_sequence_batch,batch_first=True).to(device)
        y_batch = torch.LongTensor(y_batch).to(device)
        
        optimizer.zero_grad()
        output = model(X_sequence_batch)
        loss = loss_function(output, y_batch)
        train_loss += loss.item() * output.size(0)
        count += output.size(0)
        preds = output.max(1)[1].type_as(y_batch)
        correct += torch.sum(preds.eq(y_batch).double())
        loss.backward()
        optimizer.step()
    
    #if epoch % 5 == 0:
    #print('Epoch: {:03d}'.format(epoch+1),
    #      'loss_train: {:.4f}'.format(train_loss / count),
    #      'acc_train: {:.4f}'.format(correct / count),
    #      'time: {:.4f}s'.format(time.time() - t))
    loss_train = train_loss / count
    acc_train = correct / count
    time_train = time.time() - t

    # Validate

    model.eval()
    val_loss = 0
    correct = 0
    count = 0
    # Iterate over the batches
    for i in range(0, N_val, batch_size):
        X_sequence_batch = list()
        y_batch = list()
        
        # Create tensors
        for j in range(i, min(N_val, i+batch_size)):
            X_sequence_batch.append(torch.FloatTensor(X_val_sequence[j]))
            y_batch.append(y_val[j])
            
        X_sequence_batch = pad_sequence(X_sequence_batch,batch_first=True).to(device)
        y_batch = torch.LongTensor(y_batch).to(device)
        
        output = model(X_sequence_batch)
        loss = loss_function(output, y_batch)
        val_loss += loss.item() * output.size(0)
        count += output.size(0)
        preds = output.max(1)[1].type_as(y_batch)
        correct += torch.sum(preds.eq(y_batch).double())

    loss_val = val_loss / count
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train),
          'loss_val: {:.4f}'.format(loss_val),
          'acc_train: {:.4f}'.format(acc_train),
          'acc_val: {:.4f}'.format(correct / count),
          'time_train: {:.4f}s'.format(time_train))
    if early_stopping.early_stop(model, loss_val):
      break

Epoch: 001 loss_train: 13.1231 loss_val: 2.6336 acc_train: 0.1808 acc_val: 0.2045 time_train: 19.1550s
Saving best model at: 'merged-model.pt'
Epoch: 002 loss_train: 2.5409 loss_val: 2.4620 acc_train: 0.2041 acc_val: 0.2045 time_train: 19.1646s
Saving best model at: 'merged-model.pt'
Epoch: 003 loss_train: 2.4256 loss_val: 2.3935 acc_train: 0.2041 acc_val: 0.2045 time_train: 18.2715s
Saving best model at: 'merged-model.pt'
Epoch: 004 loss_train: 2.3815 loss_val: 2.3682 acc_train: 0.2041 acc_val: 0.2045 time_train: 18.2157s
Saving best model at: 'merged-model.pt'
Epoch: 005 loss_train: 2.3651 loss_val: 2.3585 acc_train: 0.2041 acc_val: 0.2045 time_train: 18.4551s
Saving best model at: 'merged-model.pt'
Epoch: 006 loss_train: 2.3585 loss_val: 2.3544 acc_train: 0.2041 acc_val: 0.2045 time_train: 18.4783s
Saving best model at: 'merged-model.pt'
Epoch: 007 loss_train: 2.3557 loss_val: 2.3527 acc_train: 0.2041 acc_val: 0.2045 time_train: 18.3908s
Saving best model at: 'merged-model.pt'
Epoch

KeyboardInterrupt: ignored

## Other stuff

In [ ]:
class EarlyStopping:
    def __init__(self, patience=1, min_delta=1, file_name="model.pt"):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf
        self.file_name = file_name

    def early_stop(self, model, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
            print(f"Saving best model at: '{self.file_name}'")
            torch.save(model.state_dict(), self.file_name)
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
#early_stopper = EarlyStopper(patience=3, min_delta=10)
#torch.save(model.state_dict(), 'best-model-parameters.pt') # official 

# To load
#the_model = TheModelClass(*args, **kwargs)
#the_model.load_state_dict(torch.load(PATH))

In [ ]:
class SequenceEncoder(nn.Module):
    """
    Simple message passing model that consists of 2 message passing layers
    and the sum aggregation function
    """
    def __init__(self, input_dim, input_seq_dim, hidden_dim, dropout, n_class):
        super(SequenceEncoder, self).__init__()
        #self.fc1 = nn.Linear(input_dim, hidden_dim)
        #self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        #self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        #self.fc4 = nn.Linear(hidden_dim, n_class)
        self.fc_seq1 = nn.Linear(input_seq_dim, 2*hidden_dim)
        self.fc_seq2 = nn.Linear(2*hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, n_class)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_in, adj, idx, seq):

        # Processing of the sequence
        out_seq = self.relu(self.fc_seq1(seq))
        out_seq = self.dropout(out_seq)
        out_seq = self.relu(self.fc_seq2(out_seq))

        out_seq = self.bn(out_seq)

        # Merging both
        out = self.fc_out(out_seq)

        return F.log_softmax(out, dim=1)

In [ ]:
class GNNSimple(nn.Module):
    """
    Simple message passing model that consists of 2 message passing layers
    and the sum aggregation function
    """
    def __init__(self, input_dim, input_seq_dim, hidden_dim, dropout, n_class):
        super(GNNSimple, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, n_class)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_in, adj, idx, seq):
        # first message passing layer
        x = self.fc1(x_in)
        x = self.relu(torch.mm(adj, x))
        x = self.dropout(x)

        # second message passing layer
        x = self.fc2(x)
        x = self.relu(torch.mm(adj, x))
        
        # sum aggregator
        idx = idx.unsqueeze(1).repeat(1, x.size(1))
        out = torch.zeros(torch.max(idx)+1, x.size(1)).to(x_in.device)
        out = out.scatter_add_(0, idx, x)
        
        # batch normalization layer
        out = self.bn(out)

        # mlp to produce output
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.fc4(out)

        return F.log_softmax(out, dim=1)

In [ ]:
# Load graphs
adj, features, edge_features = load_data()

# Normalize adjacency matrices
adj = [normalize_adjacency(A) for A in adj]

# Split data into training and test sets
adj_train = list()
features_train = list()
y_train = list()
adj_test = list()
features_test = list()
proteins_test = list()
with open('graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            proteins_test.append(t[0])
            adj_test.append(adj[i])
            features_test.append(features[i])
        else:
            adj_train.append(adj[i])
            features_train.append(features[i])
            y_train.append(int(t[1][:-1]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train_sequence, X_val_sequence, adj_train, adj_val, features_train, features_val, y_train, y_val = train_test_split(X_train_sequence, adj_train,features_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
# Initialize device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Hyperparameters
epochs = 100 #50
batch_size = 64
n_hidden = 64
n_input = 86
dropout = 0.2
learning_rate = 1e-4 #0.001
n_class = 18

# Compute number of training and test samples
N_train = len(adj_train)
N_val = len(adj_val)
N_test = len(adj_test)

# Initializes model and optimizer
#model = GNNSimple(n_input, input_seq_dim, n_hidden, dropout, n_class).to(device)
new_model = GNN(n_input, input_seq_dim, n_hidden, dropout, n_class)

model = SequenceEncoder(n_input, input_seq_dim, n_hidden, dropout, n_class)
model.load_state_dict(torch.load("sentence-encoder.pt"))
new_model.fc_seq1 = model.fc_seq1
new_model.fc_seq2 = model.fc_seq2
new_model.bn2 = model.bn

model = GNNSimple(n_input, input_seq_dim, n_hidden, dropout, n_class)
model.load_state_dict(torch.load("gnn-simple.pt"))
new_model.fc1 = model.fc1
new_model.fc2 = model.fc2
new_model.fc3 = model.fc3
new_model.bn1 = model.bn

model = new_model
del new_model
model = model.to(device)

for n, p in model.named_parameters():
  if 'fc1' in n or 'fc2' in n or 'fc3' in n or 'fc_seq1' in n or 'fc_seq2' in n:
    p.requires_grad = False

optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=5, min_delta=0.1, file_name="merged-model.pt")

# Train model
for epoch in range(epochs):
    t = time.time()
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    # Iterate over the batches
    for i in range(0, N_train, batch_size):
        adj_batch = list()
        features_batch = list()
        idx_batch = list()
        y_batch = list()
        X_sequence_batch = list()
        
        # Create tensors
        for j in range(i, min(N_train, i+batch_size)):
            n = adj_train[j].shape[0]
            adj_batch.append(adj_train[j]+sp.identity(n))
            features_batch.append(features_train[j])
            idx_batch.extend([j-i]*n)
            y_batch.append(y_train[j])
            X_sequence_batch.append(X_train_sequence[j, :])
            
        adj_batch = sp.block_diag(adj_batch)
        features_batch = np.vstack(features_batch)
        X_sequence_batch = sp.vstack(X_sequence_batch)

        adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device)
        features_batch = torch.FloatTensor(features_batch).to(device)
        idx_batch = torch.LongTensor(idx_batch).to(device)
        y_batch = torch.LongTensor(y_batch).to(device)
        X_sequence_batch = sparse_mx_to_torch_sparse_tensor(X_sequence_batch).to(device)
        
        optimizer.zero_grad()
        output = model(features_batch, adj_batch, idx_batch, X_sequence_batch)
        loss = loss_function(output, y_batch)
        train_loss += loss.item() * output.size(0)
        count += output.size(0)
        preds = output.max(1)[1].type_as(y_batch)
        correct += torch.sum(preds.eq(y_batch).double())
        loss.backward()
        optimizer.step()
    
    #if epoch % 5 == 0:
    #print('Epoch: {:03d}'.format(epoch+1),
    #      'loss_train: {:.4f}'.format(train_loss / count),
    #      'acc_train: {:.4f}'.format(correct / count),
    #      'time: {:.4f}s'.format(time.time() - t))
    loss_train = train_loss / count
    acc_train = correct / count
    time_train = time.time() - t

    # Validate

    model.eval()
    val_loss = 0
    correct = 0
    count = 0
    # Iterate over the batches
    for i in range(0, N_val, batch_size):
        adj_batch = list()
        features_batch = list()
        idx_batch = list()
        y_batch = list()
        X_sequence_batch = list()
        
        # Create tensors
        for j in range(i, min(N_val, i+batch_size)):
            n = adj_val[j].shape[0]
            adj_batch.append(adj_val[j]+sp.identity(n))
            features_batch.append(features_val[j])
            idx_batch.extend([j-i]*n)
            y_batch.append(y_val[j])
            X_sequence_batch.append(X_val_sequence[j, :])
            
        adj_batch = sp.block_diag(adj_batch)
        features_batch = np.vstack(features_batch)
        X_sequence_batch = sp.vstack(X_sequence_batch)

        adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device)
        features_batch = torch.FloatTensor(features_batch).to(device)
        idx_batch = torch.LongTensor(idx_batch).to(device)
        y_batch = torch.LongTensor(y_batch).to(device)
        X_sequence_batch = sparse_mx_to_torch_sparse_tensor(X_sequence_batch).to(device)
        
        output = model(features_batch, adj_batch, idx_batch, X_sequence_batch)
        loss = loss_function(output, y_batch)
        val_loss += loss.item() * output.size(0)
        count += output.size(0)
        preds = output.max(1)[1].type_as(y_batch)
        correct += torch.sum(preds.eq(y_batch).double())

    loss_val = val_loss / count
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train),
          'loss_val: {:.4f}'.format(loss_val),
          'acc_train: {:.4f}'.format(acc_train),
          'acc_val: {:.4f}'.format(correct / count),
          'time_train: {:.4f}s'.format(time_train))
    if early_stopping.early_stop(model, loss_val):
      break

Epoch: 001 loss_train: 2.7279 loss_val: 2.6476 acc_train: 0.2481 acc_val: 0.3671 time_train: 5.4289s
Saving best model at: 'merged-model.pt'
Epoch: 002 loss_train: 2.3622 loss_val: 2.3951 acc_train: 0.5798 acc_val: 0.4427 time_train: 5.7735s
Saving best model at: 'merged-model.pt'
Epoch: 003 loss_train: 1.9609 loss_val: 2.1444 acc_train: 0.6706 acc_val: 0.4724 time_train: 4.6428s
Saving best model at: 'merged-model.pt'
Epoch: 004 loss_train: 1.5791 loss_val: 1.9491 acc_train: 0.7182 acc_val: 0.4969 time_train: 4.7433s
Saving best model at: 'merged-model.pt'
Epoch: 005 loss_train: 1.2859 loss_val: 1.8165 acc_train: 0.7481 acc_val: 0.5102 time_train: 4.7251s
Saving best model at: 'merged-model.pt'
Epoch: 006 loss_train: 1.0594 loss_val: 1.7239 acc_train: 0.7939 acc_val: 0.5286 time_train: 4.7345s
Saving best model at: 'merged-model.pt'
Epoch: 007 loss_train: 0.8902 loss_val: 1.6569 acc_train: 0.8330 acc_val: 0.5481 time_train: 5.7450s
Saving best model at: 'merged-model.pt'
Epoch: 008 lo

In [ ]:
model.load_state_dict(torch.load("merged-model.pt"))

<All keys matched successfully>

In [ ]:
# Evaluate model
model.eval()
y_pred_proba = list()
# Iterate over the batches
for i in range(0, N_test, batch_size):
    adj_batch = list()
    idx_batch = list()
    features_batch = list()
    y_batch = list()
    X_test_sequence_batch = list()
    
    # Create tensors
    for j in range(i, min(N_test, i+batch_size)):
        n = adj_test[j].shape[0]
        adj_batch.append(adj_test[j]+sp.identity(n))
        features_batch.append(features_test[j])
        idx_batch.extend([j-i]*n)
        X_test_sequence_batch.append(X_test_sequence[j, :])
        
    adj_batch = sp.block_diag(adj_batch)
    features_batch = np.vstack(features_batch)
    X_test_sequence_batch = sp.vstack(X_test_sequence_batch)

    adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device)
    features_batch = torch.FloatTensor(features_batch).to(device)
    idx_batch = torch.LongTensor(idx_batch).to(device)
    X_test_sequence_batch = sparse_mx_to_torch_sparse_tensor(X_test_sequence_batch).to(device)

    output = model(features_batch, adj_batch, idx_batch, X_test_sequence_batch)
    y_pred_proba.append(output)
    
y_pred_proba = torch.cat(y_pred_proba, dim=0)
y_pred_proba = torch.exp(y_pred_proba)
y_pred_proba = y_pred_proba.detach().cpu().numpy()

# Write predictions to a file
with open('sample_submission.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = list()
    for i in range(18):
        lst.append('class'+str(i))
    lst.insert(0, "name")
    writer.writerow(lst)
    for i, protein in enumerate(proteins_test):
        lst = y_pred_proba[i,:].tolist()
        lst.insert(0, protein)
        writer.writerow(lst)

In [ ]:
model.fc1

Linear(in_features=86, out_features=64, bias=True)